<h1 align="center" style="font-size: 40px; color:#2c3e50; font-weight:700;">
🛰️ Monitoreo y caracterización de cambios en coberturas agrícolas utilizando datos SAR Sentinel-1 <span style="color:#16a085;">(v3.1)</span>
</h1>

<div align="center" style="font-size:17px; color:#333; margin:10px 0;">
  <b>Autor:</b> Fredy Martínez (Fedearroz-FNA) <br>
  <b>Fecha de actualización:</b> octubre de 2025
</div>

---

## Resumen

<p align="justify">Este notebook es una herramienta diseñada para la detección, clasificación e interpretación de cambios en áreas agrícolas, a partir de imágenes de Radar de Apertura Sintética (SAR, por sus siglas en inglés) obtenidas del satélite Sentinel-1. El análisis se ejecuta en la plataforma de computación en la nube Google Earth Engine (GEE) y está optimizado para distinguir los cambios de interés (como ciclos de cultivo y cosecha) de las variaciones estacionales propias del paisaje.</p>

---

## ⚙️ Configuración inicial del cuaderno

Ejecutar esta celda una vez al inicio para:

- Cargar los estilos generales del notebook.
- Inicializar Google Earth Engine con el proyecto indicado (si aplica).
- Mostrar el formulario para configurar fechas y cargar el área de interés (GPKG).

Al finalizar la ejecución, el panel permanecerá disponible para seleccionar el archivo y las fechas requeridas.

In [11]:
#@title <font color=#88B04B>⚙️ Configuración Inicial del Notebook</font>


#@title <font color=#88B04B>⚙️ Configuración Inicial del Notebook</font>

import ee, json, os, sys, subprocess, tempfile
import ipywidgets as widgets
from datetime import date
from IPython.display import display, clear_output, HTML
# --- Setup para Colab y widgets (primera celda del notebook) ---
!pip -q install ipywidgets==7.7.1
from google.colab import output
output.enable_custom_widget_manager()

display(HTML("""
<style>
:root {
  --gee-card-bg-light: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
  --gee-card-bg-dark: linear-gradient(135deg, rgba(31, 38, 49, 0.95) 0%, rgba(17, 23, 32, 0.95) 100%);
  --gee-success-bg-light: linear-gradient(130deg, #edf9f1 0%, #d7f1e1 100%);
  --gee-success-bg-dark: linear-gradient(130deg, rgba(24, 56, 39, 0.92) 0%, rgba(19, 66, 36, 0.92) 100%);
  --gee-info-bg-light: #e3f2fd;
  --gee-info-bg-dark: rgba(33, 150, 243, 0.18);
  --gee-warning-bg-light: #fff3cd;
  --gee-warning-bg-dark: rgba(255, 193, 7, 0.18);
  --gee-error-bg-light: #ffebee;
  --gee-error-bg-dark: rgba(244, 67, 54, 0.2);

  /* NUEVO: alto controlable de la tarjeta AOI */
  --aoi-card-min-h: 120px;
}

.setup-card, .analysis-card, .monitoring-card, .multitemp-card {
  border: 2px solid;
  border-radius: 12px;
  padding: 20px;
  background: var(--gee-card-bg-light);
  margin: 10px 0;
}

.setup-card { border-color: #88B04B; }
.setup-card h3 { color: #88B04B; margin-top: 0; }

.analysis-card, .monitoring-card, .multitemp-card { border-color: #1B7192; }
.analysis-card h3, .monitoring-card h3, .multitemp-card h3 { color: #1B7192; margin-top: 0; }

/* Cajas de mensaje */
.info-box {
  background: var(--gee-info-bg-light);
  padding: 15px;
  border-radius: 8px;
  border-left: 4px solid #2196f3;
  margin: 10px 0;
}

.success-box {
  display: flex;
  gap: 14px;
  align-items: flex-start;
  background: var(--gee-success-bg-light);
  color: #1f3d2c;
  padding: 18px 22px;
  border-radius: 14px;
  border: 1px solid rgba(46, 125, 50, 0.25);
  box-shadow: 0 12px 28px rgba(31, 94, 55, 0.12);
  margin: 12px 0;
}

.success-box .icon {
  font-size: 30px;
  line-height: 1;
  margin-top: 2px;
}

.success-box .content {
  flex: 1;
}

.success-box h3 {
  margin: 0 0 6px 0;
  font-weight: 700;
  color: #1b5e38;
  font-size: 20px;
}

.success-box p {
  margin: 0 0 6px 0;
  font-size: 14px;
  line-height: 1.5;
}

.success-box .next-step span {
  font-weight: 600;
  color: #1b5e38;
}

.warning-box {
  background: var(--gee-warning-bg-light);
  padding: 12px;
  border-radius: 6px;
  border-left: 4px solid #ff9800;
}

.error-box {
  background: var(--gee-error-bg-light);
  padding: 12px;
  border-radius: 6px;
  border-left: 4px solid #f44336;
}

/* NUEVO: tarjeta AOI cargado con alto controlable */
.aoi-card {
  background: var(--gee-success-bg-light);
  border-left: 3px solid #28a745;
  border-radius: 10px;
  padding: 10px 12px;
  min-height: var(--aoi-card-min-h);
  display: grid;          /* centrado vertical */
  align-content: center;  /* centrado vertical del contenido */
  max-width: 360px;
  font-size: 13px;
  line-height: 1.4;
}
.aoi-card h4 {
  margin: 0 0 6px 0;
  color: #155724;
  font-size: 14px;
}

/* Tablas */
table {
  border-collapse: collapse;
  width: 100%;
  margin: 10px 0;
}

table td, table th {
  padding: 8px;
  text-align: left;
  border-bottom: 1px solid rgba(0, 0, 0, 0.08);
}

table tr:nth-child(even) {
  background-color: #f2f2f2;
}

.gee-note {
  color: #1d4b2c;
}

@media (prefers-color-scheme: dark) {
  .setup-card, .analysis-card, .monitoring-card, .multitemp-card {
    background: var(--gee-card-bg-dark);
    border-color: rgba(117, 201, 250, 0.55);
    box-shadow: 0 18px 32px rgba(5, 10, 18, 0.55);
  }

  .setup-card { border-color: rgba(136, 176, 75, 0.75); }
  .setup-card h3 { color: #c6f6a0; }

  .analysis-card, .monitoring-card, .multitemp-card { border-color: rgba(100, 181, 246, 0.75); }
  .analysis-card h3, .monitoring-card h3, .multitemp-card h3 { color: #9cd7ff; }

  .info-box {
    background: var(--gee-info-bg-dark);
    border-left-color: #64b5f6;
    color: #e3f4ff;
  }

  .success-box {
    background: var(--gee-success-bg-dark);
    color: #e0ffe9;
    border: 1px solid rgba(129, 199, 132, 0.45);
    box-shadow: 0 16px 40px rgba(0, 0, 0, 0.45);
  }

  .success-box h3 { color: #9ae6b4; }
  .success-box .next-step span { color: #9ae6b4; }

  /* AOI card en modo oscuro */
  .aoi-card {
    background: var(--gee-success-bg-dark);
    border-left-color: #81c784;
    color: #e0ffe9;
  }
  .aoi-card h4 { color: #9ae6b4; }

  .gee-note {
    color: #d5fbd5;
  }

  table {
    color: #e3e8f0;
  }

  table td, table th {
    border-bottom: 1px solid rgba(227, 232, 240, 0.2);
  }

  table tr:nth-child(even) {
    background-color: rgba(255, 255, 255, 0.06);
  }
}
</style>

<div class="success-box">
  <div class="icon">⚙️</div>
  <div class="content">
    <h3>Configuración lista</h3>
    <p class="next-step"><span>Paso siguiente:</span> Completa el formulario de abajo para cargar tu área de interés.</p>
  </div>
</div>
"""))

# ========== Funciones auxiliares ==========
def ensure_ee_initialized(project_id: str = None):
    """Inicializa Earth Engine con autenticación si es necesario."""
    try:
        if project_id and project_id.strip():
            ee.Initialize(project=project_id.strip())
        else:
            ee.Initialize()
    except Exception:
        _status("Se solicita autenticación de Google Earth Engine.", "warning")
        ee.Authenticate()
        if project_id and project_id.strip():
            ee.Initialize(project=project_id.strip())
        else:
            ee.Initialize()

def ensure_geo_deps():
    """Instala y retorna geopandas y fiona."""
    try:
        import geopandas as gpd
        import fiona
        return gpd, fiona
    except Exception:
        pkgs = ["geopandas>=0.14.3", "fiona>=1.9.5", "shapely>=2.0.1", "pyproj>=3.6.0"]
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet"] + pkgs)
        import geopandas as gpd
        import fiona
        return gpd, fiona

# ========== Funciones de carga de GPKG ==========
def read_gpkg_from_bytes(raw_bytes: bytes, layer_name: str = None):
    """
    Lee un GPKG desde bytes y retorna (gdf, capa_elegida, lista_capas).
    """
    gpd, fiona = ensure_geo_deps()

    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".gpkg")
    tmp_path = tmp.name
    try:
        tmp.write(raw_bytes)
        tmp.close()

        layers = fiona.listlayers(tmp_path)
        if not layers:
            raise RuntimeError("El GPKG no contiene capas")

        layer = layer_name if layer_name in layers else layers[0]
        gdf = gpd.read_file(tmp_path, layer=layer)

        return gdf, layer, layers
    finally:
        try:
            os.remove(tmp_path)
        except Exception:
            pass

def gdf_to_ee_fc(gdf):
    """Convierte GeoDataFrame a FeatureCollection de Earth Engine."""
    if gdf.crs is None:
        gdf = gdf.set_crs(4326)
    else:
        gdf = gdf.to_crs(4326)

    geojson = json.loads(gdf.to_json())
    return ee.FeatureCollection(geojson["features"])

# ========== Variables globales ==========
CONFIG = {}
AOI_FC = None
AOI_LAYER_NAME = None
RAW_GPKG_BYTES = None
IS_COLAB = 'google.colab' in sys.modules

# ========== Widgets ==========
style = {'description_width': '140px'}

w_project = widgets.Text(
    description="GEE Project ID:",
    placeholder="(opcional) my-project-123",
    style=style,
    layout=widgets.Layout(width="100%")
)

today = date.today()
w_fecha_ini = widgets.DatePicker(
    description="Fecha inicio:",
    value=date(today.year, 1, 1),
    style=style
)
w_fecha_fin = widgets.DatePicker(
    description="Fecha fin:",
    value=today,
    style=style
)

w_file_label = widgets.HTML(
    "<div style='color:#666; font-size:12px; padding:5px 0;'>No hay archivo seleccionado</div>"
)

w_layer_dd = widgets.Dropdown(
    description="Capa:",
    options=[],
    disabled=True,
    style=style,
    layout=widgets.Layout(width="50%")
)

w_btn = widgets.Button(
    description="✓ Cargar y Procesar",
    button_style="success",
    icon="check",
    layout=widgets.Layout(width="auto", margin="10px 0")
)

w_output = widgets.Output()

# Selección de archivo: usar FileUpload en Jupyter clásico y botón en Colab
if IS_COLAB:
    w_file = widgets.Button(
        description="📁 Seleccionar GPKG",
        button_style="info",
        icon="upload",
        layout=widgets.Layout(width="auto")
    )
else:
    w_file = widgets.FileUpload(
        accept=".gpkg,.GPKG",
        multiple=False,
        description="📁 Archivo GPKG:",
        style=style,
        layout=widgets.Layout(width="100%")
    )

def _status(message: str, kind: str = 'info'):
    colors = {
        'info': '#3498db',
        'warning': '#f39c12',
        'error': '#c0392b',
        'success': '#27ae60'
    }
    color = colors.get(kind, '#3498db')
    with w_output:
        display(HTML(f"<div style='color:{color}; margin:4px 0;'>{message}</div>"))

def _reset_file_ui(message: str = "No hay archivo seleccionado", color: str = '#666'):
    w_file_label.value = f"<div style='color:{color}; font-size:12px; padding:5px 0;'>{message}</div>"
    w_layer_dd.options = []
    w_layer_dd.disabled = True

def _process_uploaded_file(fname, raw, metadata=None):
    global RAW_GPKG_BYTES

    if metadata is None:
        metadata = {}
    elif not isinstance(metadata, dict):
        metadata = dict(metadata)

    if fname is None:
        fname = metadata.get('name')
    fname = str(fname or 'archivo.gpkg')

    if raw is None:
        _reset_file_ui("❌ Error al leer el archivo", '#c0392b')
        return False

    if isinstance(raw, str):
        raw = raw.encode('utf-8')
    elif isinstance(raw, memoryview):
        raw = raw.tobytes()
    elif hasattr(raw, 'tobytes'):
        try:
            raw = raw.tobytes()
        except Exception:
            pass
    if raw is not None and not isinstance(raw, (bytes, bytearray)):
        try:
            raw = bytes(raw)
        except Exception:
            raw = None

    if raw is None:
        _reset_file_ui("❌ Error al leer el archivo", '#c0392b')
        return False

    if not fname.lower().endswith('.gpkg'):
        _reset_file_ui("⚠️ Debe ser .gpkg", '#c0392b')
        return False

    RAW_GPKG_BYTES = raw

    try:
        gdf, selected_layer, layers = read_gpkg_from_bytes(raw)
    except Exception as err:
        RAW_GPKG_BYTES = None
        _reset_file_ui(f"❌ {err}", '#c0392b')
        return False

    w_layer_dd.options = layers
    w_layer_dd.value = selected_layer
    w_layer_dd.disabled = False
    w_file_label.value = (
        f"<div style='color:#27ae60; font-size:12px;'>✓ {fname} ({len(layers)} capas)</div>"
    )
    _status(f"El archivo {fname} se cargó correctamente.", 'success')
    return True

def on_file_change(change):
    if change.get('name') != 'value':
        return

    w_output.clear_output(wait=True)

    def _first_upload(value):
        if isinstance(value, dict):
            return next(iter(value.values()), None)
        if isinstance(value, (tuple, list)):
            return value[0] if value else None
        return value

    entry = _first_upload(w_file.value)

    if entry is None and IS_COLAB:
        return  # En Colab se maneja mediante el botón dedicado

    if entry is None:
        _reset_file_ui()
        return

    if isinstance(entry, dict):
        metadata = entry.get('metadata', {}) or {}
        fname = metadata.get('name', entry.get('name'))
        raw = entry.get('content') or entry.get('bytes')
    else:
        metadata = getattr(entry, 'metadata', {}) or {}
        fname = metadata.get('name', getattr(entry, 'name', None))
        raw = getattr(entry, 'content', None)

    _process_uploaded_file(fname, raw, metadata)

def on_colab_upload(_):
    w_output.clear_output(wait=True)
    try:
        from google.colab import files as _colab_files  # type: ignore
    except Exception as err:
        _status(f"No se pudo acceder a google.colab.files: {err}", 'error')
        _reset_file_ui()
        return

    _status("Seleccionar el archivo GPKG desde el cuadro de diálogo.", 'info')
    uploaded = _colab_files.upload()
    if not uploaded:
        _reset_file_ui()
        return

    name, raw = next(iter(uploaded.items()))
    _process_uploaded_file(name, raw, {'name': name})

def on_button_click(_):
    """Procesa el archivo GPKG y carga en GEE."""
    global CONFIG, AOI_FC, AOI_LAYER_NAME

    w_output.clear_output(wait=True)

    if w_fecha_ini.value is None or w_fecha_fin.value is None:
        _status("⚠️ Fechas inválidas", 'warning')
        return

    if w_fecha_ini.value > w_fecha_fin.value:
        _status("⚠️ Fecha inicio > fecha fin", 'warning')
        return

    if RAW_GPKG_BYTES is None:
        _status("⚠️ Debe seleccionar un archivo GPKG", 'warning')
        return

    _status("⏳ Inicializando Google Earth Engine...", 'info')
    try:
        ensure_ee_initialized(w_project.value)
        ee.Number(1).getInfo()
    except Exception as err:
        _status(f"❌ Error GEE: {err}", 'error')
        return

    _status("⏳ Procesando GPKG...", 'info')
    try:
        layer_name = w_layer_dd.value if w_layer_dd.options else None
        gdf, capa, _ = read_gpkg_from_bytes(RAW_GPKG_BYTES, layer_name)
        fc = gdf_to_ee_fc(gdf)
    except Exception as err:
        _status(f"❌ Error al procesar el GPKG: {err}", 'error')
        return

    CONFIG.update({
        "gee_project_id": (w_project.value or '').strip() or None,
        "fecha_inicio": w_fecha_ini.value.isoformat(),
        "fecha_fin": w_fecha_fin.value.isoformat(),
        "gpkg_layer": capa,
    })
    AOI_FC = fc
    AOI_LAYER_NAME = capa

    try:
        count = AOI_FC.size().getInfo()
    except Exception:
        count = "?"

    # ====== NUEVO: caja AOI usando clase CSS con altura controlable ======
    summary_html = f"""
    <div class="aoi-card">
        <h4>✓ AOI cargado exitosamente</h4>
        <table style="width:100%; border-collapse:collapse;">
            <tr><td style="padding:2px 0;"><b>Capa:</b></td><td style="padding:2px 0;">{AOI_LAYER_NAME}</td></tr>
            <tr><td style="padding:2px 0;"><b>Features:</b></td><td style="padding:2px 0;">{count}</td></tr>
            <tr><td style="padding:2px 0;"><b>Período:</b></td><td style="padding:2px 0;">{CONFIG['fecha_inicio']} → {CONFIG['fecha_fin']}</td></tr>
        </table>
    </div>
    """
    with w_output:
        display(HTML(summary_html))
    _status("El área de interés queda cargada y lista para su uso.", 'success')

if IS_COLAB:
    w_file.on_click(on_colab_upload)
else:
    w_file.observe(on_file_change, names='value')

_reset_file_ui()

w_btn.on_click(on_button_click)

# ========== UI ==========
panel = widgets.VBox([
    widgets.HTML("<h3 style='color:#88B04B;'>📍 Configuración del Área de Interés</h3>"),
    w_project,
    widgets.HTML("<hr style='margin:10px 0;'>"),
    widgets.HTML("<h4>Rango Temporal</h4>"),
    widgets.HBox([w_fecha_ini, w_fecha_fin]),
    widgets.HTML("<hr style='margin:10px 0;'>"),
    widgets.HTML("<h4>Archivo Geopackage</h4>"),
    w_file,
    w_file_label,
    w_layer_dd,
    w_btn,
    w_output
], layout=widgets.Layout(
    border='2px solid #88B04B',
    padding='20px',
    border_radius='8px',
    margin='10px 0'
))

display(panel)

Saving Yopal.gpkg to Yopal (1).gpkg


## 📚 Guía de uso y conceptos clave

La siguiente guía resume los pasos y conceptos fundamentales para trabajar con el notebook y con los datos de Sentinel-1 orientados al monitoreo del cultivo de arroz.

### 🎯 Configuración del área de interés

- **Fecha de inicio.** Corresponde a la siembra esperada. El cuaderno utiliza esta fecha para definir automáticamente los rangos de análisis.
- **Fecha de fin.** Se recomienda cubrir al menos 150 días posteriores a la fecha de inicio para abarcar un ciclo completo del cultivo.
- **Área de interés (AOI).** Una vez cargado el GPKG, se habilita la selección de la capa y se confirma la cantidad de entidades detectadas. Se aconseja revisar que la geometría corresponda exactamente al lote o bloque de análisis.

### 📊 Interpretación de los resultados

#### Valor p

- Valores **cercanos a 0** sugieren cambios estadísticamente significativos y, por tanto, alta probabilidad de que exista un evento real.
- Valores **cercanos a 1** indican que el comportamiento observado es compatible con ruido o variaciones naturales.
- El cuaderno emplea un umbral $(\alpha = 0.01)$, lo que equivale a un 99 % de confianza y minimiza falsos positivos, algo especialmente útil en arroz donde las variaciones de la señal SAR son marcadas.

#### Mapas de dirección del cambio (orden de Loewner)

| Color | Interpretación | Señal típica en arroz |
| --- | --- | --- |
| 🟢 **Positivo definido** | Incremento de VV y VH asociado a siembra o crecimiento vegetativo | ΔVH: +3 a +5 dB, ΔVV: +1 a +2 dB |
| 🔴 **Negativo definido** | Disminución de VV y VH compatible con cosecha o inundación | ΔVH: −4 a −7 dB, ΔVV: −2 a −4 dB |
| 🟠 **Indefinido** | Cambios mixtos; se recomienda verificación adicional | Depende del contexto |

#### Mapas multitemporales

- **Frecuencia de cambios (fmap).** Mide cuántas transiciones significativas experimenta cada píxel. Valores entre 2 y 4 suelen corresponder a parcelas de arroz.
- **Primer cambio (smap).** Indica cuándo ocurrió la primera transición relevante. Los colores fríos representan siembras tempranas y los cálidos, siembras tardías.
- **Cambio más reciente (cmap).** Refleja la última transición detectada y sirve como aproximación a la fecha de cosecha.

### ⚙️ Parámetros técnicos

- **Polarizaciones VV y VH.** Se analizan de forma conjunta para aprovechar la sensibilidad de VV a superficies lisas y la mayor respuesta de VH ante estructuras vegetales.
- **Intervalo temporal.** El valor por defecto es 12 días (revisita de Sentinel-1A). Si se dispone de Sentinel-1B, puede reducirse a 6 días.
- **ENL (equivalent number of looks).** Se fija en 4.4 para imágenes Sentinel-1 IW, de acuerdo con la documentación de ESA.
- **Filtro mediano.** Atenúa el ruido salt-and-pepper y se recomienda mantenerlo activo.

### 💡 Recomendaciones prácticas

1. **Cruce con imágenes ópticas.** Comparar los resultados con Sentinel-2 o datos ópticos disponibles ayuda a confirmar cambios ambiguos.
2. **Validación local.** La información de agrónomos, técnicos o productores aporta contexto sobre fechas reales de siembra y cosecha.
3. **Datos de campo.** Registros de parcelas (variedad, fecha de siembra, manejo) refuerzan la interpretación de los mapas.
4. **Considerar el calendario agrícola.** Ajustar los análisis a la temporada (lluvias, riego) y a la disponibilidad hídrica de la región.
5. **Reconocer limitaciones.** Parcelas menores a 1 ha, terreno con pendiente pronunciada o humedales naturales pueden requerir inspección adicional y ajustes manuales.


## Metodología

Este notebook implementa **dos métodos** de detección de cambios con Sentinel-1:

### 1. **Método LRT simple (likelihood ratio test)**
Test de razón de verosimilitud bivariado que compara dos períodos temporales. Basado en Conradsen et al. (2016), analiza las bandas VV y VH simultáneamente para detectar cambios estadísticamente significativos. Incluye:

- Test estadístico chi-cuadrado con 2 grados de libertad
- Cálculo de valores p para determinar significancia
- Clasificación de cambios usando el **orden de Loewner** (orden de definición de matrices):
  - **Positivo definido**: Aumento en ambas polarizaciones (crecimiento vegetativo)
  - **Negativo definido**: Disminución en ambas polarizaciones (cosecha/inundación)
  - **Indefinido**: Cambio mixto

### 2. **Método multitemporal secuencial**
Implementación del **test omnibus secuencial** descrito en Conradsen et al. (2016) para series temporales completas. Este método:

- Descompone el test omnibus en productos de tests independientes $Q_k = \prod_{j=2}^k R_j$
- Detecta múltiples cambios a lo largo de la serie temporal
- Calcula mapas temáticos:
  - **cmap**: Intervalo del cambio más reciente
  - **smap**: Intervalo del primer cambio
  - **fmap**: Frecuencia/número total de cambios
  - **bmap**: Cambios por intervalo con dirección (orden de Loewner)
- Opción de filtro de mediana para reducir ruido salt-and-pepper

**ENL (equivalent number of looks)**: Se utiliza 4.4 para imágenes Sentinel-1 IW mode, según la documentación de ESA.

---

## 1️⃣ Comparación rápida de composiciones VV/VH

Esta sección permite contrastar, de forma visual, dos fechas seleccionadas por la persona usuaria. Se generan composiciones VV y VH, junto con el producto VH·VV, y se muestran en un visor tipo cortina que facilita la comparación entre ambas fechas.

> **Nota:** se recomienda ejecutar antes la sección de configuración del cuaderno para contar con el área de interés (AOI) y la autenticación de Google Earth Engine.

In [12]:
#@title <font color=#88B04B>1️⃣ Comparación rápida VV/VH</font> { display-mode: "form" }
#@markdown Selecciona dos fechas y pulsa el botón para comparar las composiciones Sentinel-1 (VV, VH y VH·VV) en un visor tipo cortina.

import sys
import subprocess
from datetime import date, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

try:
    import geemap
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "geemap>=0.33.2"])
    import geemap  # noqa

import ee
from ipyleaflet import WidgetControl

style_dates = {'description_width': '120px'}
widget_layout = widgets.Layout(width='260px', min_width='220px')

hoy = date.today()
default_inicial = date(hoy.year, 1, 15)
default_final = date(hoy.year, 9, 15)

w_fecha_a = widgets.DatePicker(description="Fecha A:", value=default_inicial, style=style_dates, layout=widget_layout)
w_fecha_b = widgets.DatePicker(description="Fecha B:", value=default_final, style=style_dates, layout=widget_layout)
w_ventana = widgets.IntSlider(description="Ventana ± días:", min=5, max=45, value=15, step=5,
                              style={'description_width': '140px'}, layout=widgets.Layout(width='300px'))

descripcion = widgets.HTML("<p class='gee-note'>Las composiciones utilizan VV, VH y el producto VH*VV. La ventana define el rango temporal de cada compuesto alrededor de la fecha indicada.</p>")

w_generar = widgets.Button(description="🔍 Generar comparación", button_style='success', layout=widgets.Layout(width='220px'))
w_mensaje = widgets.Output()

controles = widgets.VBox([
    widgets.HTML("<b>Parámetros de comparación</b>"),
    widgets.HBox([w_fecha_a, w_fecha_b]),
    w_ventana,
    descripcion,
    w_generar,
    w_mensaje
])

def _obtener_aoi():
    try:
        return AOI_FC.geometry()
    except NameError:
        return None


def _composite_s1(aoi, fecha_central, ventana_dias):
    inicio = fecha_central - timedelta(days=ventana_dias)
    fin = fecha_central + timedelta(days=ventana_dias)

    coleccion = (ee.ImageCollection('COPERNICUS/S1_GRD')
                 .filterBounds(aoi)
                 .filterDate(inicio.isoformat(), fin.isoformat())
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

    asc = coleccion.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    desc = coleccion.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    coleccion = ee.ImageCollection(ee.Algorithms.If(asc.size().gt(desc.size()), asc, desc))

    imagen = ee.Image(coleccion.median()).select(['VV', 'VH'])
    producto = imagen.select('VH').multiply(imagen.select('VV')).rename('VH_mul_VV')
    return imagen.addBands(producto)


def _visualizar_comparacion(_):
    with w_mensaje:
        clear_output()

        if w_fecha_a.value is None or w_fecha_b.value is None:
            display(HTML("<div style='color:#d93025;'>Es necesario indicar ambas fechas para continuar.</div>"))
            return

        aoi = _obtener_aoi()
        if aoi is None:
            display(HTML("<div style='color:#d93025;'>El área de interés no está disponible. Ejecute primero la sección de configuración.</div>"))
            return

        try:
            center = aoi.centroid(30).coordinates().getInfo()[::-1]
        except Exception:
            center = [0, 0]

        try:
            comp_a = _composite_s1(aoi, w_fecha_a.value, w_ventana.value)
            comp_b = _composite_s1(aoi, w_fecha_b.value, w_ventana.value)
        except Exception as err:
            display(HTML(f"<div style='color:#d93025;'>No fue posible generar los compuestos: {err}</div>"))
            return

        comp_a = comp_a.clip(aoi)
        comp_b = comp_b.clip(aoi)

        vis_params = {
            'bands': ['VH', 'VV', 'VH_mul_VV'],
            'min': [-25, -20, 0.0],
            'max': [0, -5, 625.0],
            'gamma': 1.1
        }

        # Mapa split para comparar las fechas con un control deslizante
        left_layer = geemap.ee_tile_layer(comp_a, vis_params, f"Fecha A: {w_fecha_a.value.isoformat()}")
        right_layer = geemap.ee_tile_layer(comp_b, vis_params, f"Fecha B: {w_fecha_b.value.isoformat()}")

        mapa_split = geemap.Map(center=center, zoom=11, use_google_maps=False)
        mapa_split.add_basemap('SATELLITE')
        mapa_split.split_map(left_layer, right_layer)
        mapa_split.addLayer(aoi, {'color': '#1d8043', 'fillColor': '00000000'}, 'AOI', True, 0.8)

        label_style = "background:#3a3f47; color:#f5f7fa; padding:4px 12px; border-radius:6px; border:1px solid rgba(255,255,255,0.25); font-size:13px;"
        left_note = widgets.HTML(value=f"<div style='{label_style}'>Fecha A: {w_fecha_a.value.isoformat()}</div>")
        right_note = widgets.HTML(value=f"<div style='{label_style}'>Fecha B: {w_fecha_b.value.isoformat()}</div>")
        mapa_split.add_control(WidgetControl(widget=left_note, position='bottomleft'))
        mapa_split.add_control(WidgetControl(widget=right_note, position='bottomright'))
        display(mapa_split)

w_generar.on_click(_visualizar_comparacion)

display(controles)


## 2️⃣ Análisis de cambio bitemporal

Esta etapa compara dos intervalos temporales mediante el test LRT bivariado (VV y VH). Resulta útil para validar si la señal SAR evidencia un cambio puntual entre dos cortes.

**Cómo proceder**
1. Ejecutar la celda siguiente para mostrar el formulario.
2. Seleccionar las fechas de referencia (baseline) y las fechas objetivo.
3. Revisar el encabezado: se indica el nombre de la capa cargada para confirmar que corresponde al AOI correcto.
4. Pulsar `🔎 Ejecutar análisis` para generar los mapas y las estadísticas.
5. Explorar el mapa interactivo: la capa «Valor p» muestra la significancia estadística y la capa «Dirección del cambio» indica si el cambio es positivo, negativo o mixto.

**Interpretación**
- Valores de color verde implican incremento en VV/VH (crecimiento vegetativo).
- Valores rojos indican disminución (cosecha o pérdida de biomasa).
- Valores naranjas representan comportamientos mixtos y suelen requerir verificación adicional.


In [13]:
#@title <font color=#88B04B>Análisis de cambio bitemporal</font>

import sys
import subprocess
from datetime import date
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import ee
from collections import OrderedDict

# --- Dependencias auxiliares ---
def _pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet"] + pkgs)

def ensure_geemap():
    try:
        import geemap  # noqa
    except Exception:
        _pip_install(["geemap>=0.33.2"])
        import geemap  # noqa
    return geemap

geemap = ensure_geemap()

# --- Widgets de configuración ---
style_dates = {'description_width': '120px'}
date_layout = widgets.Layout(width='260px', min_width='240px')

ref_start_picker    = widgets.DatePicker(description="Ref. inicio:", value=date(2025, 1, 1),  style=style_dates, layout=date_layout)
ref_end_picker      = widgets.DatePicker(description="Ref. fin:",    value=date(2025, 6, 30), style=style_dates, layout=date_layout)
target_start_picker = widgets.DatePicker(description="Obj. inicio:", value=date(2025, 7, 1),  style=style_dates, layout=date_layout)
target_end_picker   = widgets.DatePicker(description="Obj. fin:",    value=date(2025, 12, 31), style=style_dates, layout=date_layout)

run_button = widgets.Button(
    description="🔎 Ejecutar análisis",
    button_style="info",
    icon="play",
    layout=widgets.Layout(width='auto', margin='14px 0 0 0')
)
output_widget = widgets.Output(layout=widgets.Layout(margin='14px 0 0 0'))

try:
    aoi_label = f"<span class='gee-note'>Área de interés: <b>{AOI_LAYER_NAME}</b></span>"
except NameError:
    aoi_label = "<span style='color:#ffb4ab;'>Área de interés no cargada.</span>"

panel_control = widgets.VBox([
    widgets.HTML("<h3 style='color:#1d8043;'>🔎 Análisis de cambio bitemporal (LRT)</h3>"),
    widgets.HTML(f"<p class='gee-note' style='margin:0 0 12px 0;'>{aoi_label}</p>"),
    widgets.HTML("<p class='gee-note' style='margin:0 0 12px 0;'><strong>Ayuda:</strong> Se comparan períodos de referencia y objetivo para identificar cambios estadísticamente significativos.</p>"),
    widgets.HTML("<h4 style='margin:0 0 8px 0;'>Selección de períodos</h4>"),
    widgets.HBox([ref_start_picker, ref_end_picker], layout=widgets.Layout(margin='0 0 10px 0', flex_flow='row wrap', justify_content='flex-start', gap='12px')),
    widgets.HBox([target_start_picker, target_end_picker], layout=widgets.Layout(margin='0 0 10px 0', flex_flow='row wrap', justify_content='flex-start', gap='12px')),
    run_button,
    output_widget
])
panel_control.add_class('analysis-card')
display(panel_control)

# --- Utilidades internas ---
def _status(message, kind='info'):
    colors = {
        'info': '#1f6feb',
        'warning': '#b58105',
        'error': '#d93025',
        'success': '#1a7f37'
    }
    background = {
        'info': '#eff4fb',
        'warning': '#fbf4e2',
        'error': '#fdecea',
        'success': '#edf8f0'
    }
    color = colors.get(kind, '#1f6feb')
    bg = background.get(kind, '#eff4fb')
    display(HTML(
        f"<div style='margin:4px 0; padding:10px 12px; border-radius:8px; "
        f"border-left:4px solid {color}; background:{bg}; color:{color}; font-size:13px;'>"
        f"{message}</div>"
    ))

def preprocess_s1(aoi, start_date, end_date):
    """Crea un compuesto mediano VV/VH en formato FLOAT."""
    collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                  .filterBounds(aoi)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.eq('instrumentMode', 'IW'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

    asc = collection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    desc = collection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    use = ee.ImageCollection(ee.Algorithms.If(asc.size().gt(desc.size()), asc, desc))
    return use.median().select(['VV', 'VH'])

def det(im):
    """Determinante de matriz diagonal 2x2."""
    return im.expression('b(0) * b(1)')

def chi2cdf(chi2, df):
    """CDF Chi-cuadrado usando la gamma incompleta de GEE."""
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def bivariate_lrt(im1, im2, m=4.4):
    """Retorna (p-value, -2logQ) para el test LRT bivariado."""
    m2logQ = (det(im1).log().add(det(im2).log())
              .subtract(det(im1.add(im2)).log().multiply(2))
              .add(4 * 0.693147)
              .multiply(-2 * m))
    p_value = ee.Image.constant(1).subtract(chi2cdf(m2logQ, 2))
    return p_value, m2logQ

def run_analysis(_):
    with output_widget:
        clear_output(wait=True)
        _status("🚀 El análisis LRT bivariado se ha iniciado.", 'info')

        try:
            aoi = AOI_FC.geometry()
        except Exception:
            _status("Es necesario cargar un archivo .gpkg en la celda de configuración antes de avanzar.", 'error')
            return

        if (ref_start_picker.value is None or ref_end_picker.value is None or
                target_start_picker.value is None or target_end_picker.value is None):
            _status("Se debe completar cada rango de fechas antes de continuar.", 'warning')
            return

        ref_start = ref_start_picker.value.strftime('%Y-%m-%d')
        ref_end = ref_end_picker.value.strftime('%Y-%m-%d')
        tar_start = target_start_picker.value.strftime('%Y-%m-%d')
        tar_end = target_end_picker.value.strftime('%Y-%m-%d')

        _status("⏳ Se están generando los compuestos Sentinel-1 VV/VH...", 'info')
        try:
            ref_comp = preprocess_s1(aoi, ref_start, ref_end)
            tar_comp = preprocess_s1(aoi, tar_start, tar_end)
        except Exception as e:
            _status(f"No fue posible preparar los compuestos: {e}", 'error')
            return

        _status("📉 Se calculan los estadísticos del test LRT...", 'info')
        try:
            p_value, _ = bivariate_lrt(ref_comp, tar_comp)
            change_mask = p_value.lt(0.01)
            diff = tar_comp.subtract(ref_comp)
            d_map = change_mask.multiply(0)
            d_map = d_map.where(det(diff).gt(0), 2)
            d_map = d_map.where(diff.select(0).gt(0), 3)
            d_map = d_map.where(det(diff).lt(0), 1)
        except Exception as e:
            _status(f"No fue posible calcular el estadístico LRT: {e}", 'error')
            return

        _status("🗺️ Se renderizan los mapas interactivos...", 'info')
        try:
            center_ll = aoi.centroid(30).coordinates().getInfo()[::-1]
        except Exception:
            center_ll = [0, 0]

        Map = geemap.Map(center=center_ll, zoom=10, use_google_maps=False)
        Map.add_basemap('SATELLITE')
        Map.addLayer(aoi, {'color': '#1B7192', 'fillColor': '#00000000'}, 'Área de interés', True, 0.8)
        Map.addLayer(p_value.clip(aoi), {'min': 0, 'max': 1, 'palette': ['#1B7192', '#86c5da', '#f4e285']}, 'Valor p')
        Map.addLayer(d_map.updateMask(d_map.gt(0)).clip(aoi),
                     {'min': 1, 'max': 3, 'palette': ['#ffa600', '#ff4d4d', '#3cb371']},
                     'Dirección del cambio', True)

        legend_dict = OrderedDict([
            ('Cambio mixto (indef.)', '#ffa600'),
            ('Disminución (neg. def.)', '#ff4d4d'),
            ('Aumento (pos. def.)', '#3cb371')
        ])
        Map.add_legend(title="Dirección del cambio", legend_dict=legend_dict)

        _status("🧮 Se calculan las métricas de área...", 'info')
        try:
            area_img = ee.Image.pixelArea().updateMask(change_mask)
            stats = area_img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=aoi,
                scale=30,
                maxPixels=1e11,
                tileScale=8,
                bestEffort=True
            )
            area_cambio_m2 = ee.Number(stats.get('area', 0)).getInfo()
        except Exception as e:
            _status(f"No fue posible calcular el área de cambio: {e}", 'warning')
            area_cambio_m2 = 0

        try:
            area_total_m2 = ee.Number(aoi.area()).getInfo()
        except Exception:
            area_total_m2 = 0

        area_cambio_ha = area_cambio_m2 / 10000.0
        area_total_ha = area_total_m2 / 10000.0
        pct_cambio = (area_cambio_ha / area_total_ha * 100.0) if area_total_ha > 0 else 0.0

        summary_html = f"""
        <div class='success-box'>
          <div class='icon'>📊</div>
          <div class='content'>
            <h3>Resultados del LRT bivariado</h3>
            <p><strong>Período de referencia:</strong> {ref_start} → {ref_end}</p>
            <p><strong>Período objetivo:</strong> {tar_start} → {tar_end}</p>
            <p><strong>Área total:</strong> {area_total_ha:,.2f} ha</p>
            <p class='next-step'><span>Área con cambios:</span> {area_cambio_ha:,.2f} ha ({pct_cambio:.2f}%).</p>
          </div>
        </div>
        """

        display(HTML(summary_html))
        display(Map)
        _status("<strong>Ayuda:</strong> Se recomienda alternar las capas del mapa para examinar mejor los resultados.", 'info')

run_button.on_click(run_analysis)


## 3️⃣ Monitoreo de series temporales

La celda siguiente arma compuestos VV/VH para un período de referencia y otro período objetivo, permitiendo analizar la diferencia (ΔVV y ΔVH) de forma visual.

**Cómo proceder**
1. Ejecutar la celda para desplegar el formulario de fechas.
2. Definir los rangos de referencia y objetivo (los valores sugeridos se adaptan a un ciclo de arroz de ~6 meses).
3. Pulsar `📈 Ejecutar visualización` para generar los compuestos y la capa diferencial.
4. Examinar el mapa interactivo: la leyenda de Δ indica dónde aumentó o disminuyó la señal de radar.

**Interpretación**
- Valores positivos (tonos azulados) sugieren mayor retrodispersión en el período objetivo, típico de crecimiento vegetativo.
- Valores negativos (tonos rojizos) sugieren reducción de retrodispersión (posible cosecha o inundación).
- Se recomienda alternar la visibilidad de ΔVV y ΔVH para diferenciar cambios estructurales de cambios superficiales.


In [14]:
#@title <font color=#88B04B>Monitoreo de series temporales</font>

"""Panel interactivo para monitoreo de series temporales."""
import sys
import subprocess
from datetime import date, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import ee
from collections import OrderedDict

def _pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet"] + pkgs)

try:
    import geemap
except Exception:
    _pip_install(["geemap>=0.33.2"])
    import geemap

style_dates = {'description_width': '120px'}
date_layout = widgets.Layout(width='260px', min_width='240px')

# Calcular fechas óptimas basadas en CONFIG si existe
try:
    config_inicio = date.fromisoformat(CONFIG['fecha_inicio'])
    # Parámetros optimizados para arroz: Ref 30 días antes, Obj días 15-45 post-siembra
    ref_start_default = config_inicio - timedelta(days=30)
    ref_end_default = config_inicio
    tar_start_default = config_inicio + timedelta(days=15)
    tar_end_default = config_inicio + timedelta(days=45)
except (NameError, KeyError):
    # Valores por defecto si no hay CONFIG
    today = date.today()
    ref_start_default = date(today.year, 1, 1)
    ref_end_default = date(today.year, 1, 31)
    tar_start_default = date(today.year, 2, 15)
    tar_end_default = date(today.year, 3, 15)

ref_start = widgets.DatePicker(description='Ref. inicio:', value=ref_start_default, style=style_dates, layout=date_layout)
ref_end   = widgets.DatePicker(description='Ref. fin:',   value=ref_end_default, style=style_dates, layout=date_layout)
tar_start = widgets.DatePicker(description='Obj. inicio:', value=tar_start_default, style=style_dates, layout=date_layout)
tar_end   = widgets.DatePicker(description='Obj. fin:',    value=tar_end_default, style=style_dates, layout=date_layout)

run_btn = widgets.Button(
    description='📈 Ejecutar visualización',
    button_style='info',
    icon='line-chart',
    layout=widgets.Layout(width='auto', margin='14px 0 0 0')
)
output = widgets.Output(layout=widgets.Layout(margin='14px 0 0 0'))

panel = widgets.VBox([
    widgets.HTML("<h3 style='color:#1d8043;'>📈 Visualización temporal básica</h3>"),
    widgets.HTML("<p class='gee-note' style='margin:0 0 12px 0;'><strong>Ayuda:</strong> Se generan compuestos mediana VV y VH para visualizar diferencias entre períodos.</p>"),
    widgets.HBox([ref_start, ref_end], layout=widgets.Layout(margin='0 0 10px 0', flex_flow='row wrap', gap='12px')),
    widgets.HBox([tar_start, tar_end], layout=widgets.Layout(margin='0 0 10px 0', flex_flow='row wrap', gap='12px')),
    run_btn,
    output
])
panel.add_class('monitoring-card')
display(panel)

def _status(message, kind='info'):
    colors = {
        'info': '#1f6feb',
        'warning': '#b58105',
        'error': '#d93025',
        'success': '#1a7f37'
    }
    background = {
        'info': '#eff4fb',
        'warning': '#fbf4e2',
        'error': '#fdecea',
        'success': '#edf8f0'
    }
    color = colors.get(kind, '#1f6feb')
    bg = background.get(kind, '#eff4fb')
    display(HTML(
        f"<div style='margin:4px 0; padding:10px 12px; border-radius:8px; "
        f"border-left:4px solid {color}; background:{bg}; color:{color}; font-size:13px;'>"
        f"{message}</div>"
    ))
    return
    css_class = classes.get(kind, 'info-box')
    display(HTML(f"<div class='{css_class}'>{message}</div>"))

def run_viz(_):
    with output:
        clear_output(wait=True)
        _status('🚀 Se prepara la visualización temporal.', 'info')

        try:
            aoi = AOI_FC.geometry()
        except Exception:
            _status('Es necesario cargar un archivo .gpkg en la celda de configuración antes de avanzar.', 'error')
            return

        if None in (ref_start.value, ref_end.value, tar_start.value, tar_end.value):
            _status('Se debe completar cada rango de fechas antes de continuar.', 'warning')
            return

        def get_composite(start, end):
            collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                          .filterBounds(aoi)
                          .filterDate(start, end)
                          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                          .filter(ee.Filter.eq('instrumentMode', 'IW')))
            asc = collection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
            desc = collection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
            use = ee.ImageCollection(ee.Algorithms.If(asc.size().gt(desc.size()), asc, desc))
            return use.median().select(['VV', 'VH'])

        ref_start_str = ref_start.value.strftime('%Y-%m-%d')
        ref_end_str = ref_end.value.strftime('%Y-%m-%d')
        tar_start_str = tar_start.value.strftime('%Y-%m-%d')
        tar_end_str = tar_end.value.strftime('%Y-%m-%d')

        _status('⏳ Se están generando los compuestos de referencia y objetivo...', 'info')
        try:
            ref_comp = get_composite(ref_start_str, ref_end_str)
            tar_comp = get_composite(tar_start_str, tar_end_str)
        except Exception as e:
            _status(f'No fue posible generar la serie temporal: {e}', 'error')
            return

        _status('🎯 Se calculan las diferencias VV y VH...', 'info')
        diff_vv = tar_comp.select('VV').subtract(ref_comp.select('VV'))
        diff_vh = tar_comp.select('VH').subtract(ref_comp.select('VH'))

        try:
            center_ll = aoi.centroid(30).coordinates().getInfo()[::-1]
        except Exception:
            center_ll = [0, 0]

        Map = geemap.Map(center=center_ll, zoom=10, use_google_maps=False)
        Map.add_basemap('SATELLITE')
        Map.addLayer(aoi, {'color': '#1B7192', 'fillColor': '#00000000'}, 'Área de interés', True, 0.8)
        palette_diff = ['#c0392b', '#fdf6e3', '#1B7192']
        Map.addLayer(diff_vv.clip(aoi), {'min': -6, 'max': 6, 'palette': palette_diff}, 'Δ VV (dB)', True)
        Map.addLayer(diff_vh.clip(aoi), {'min': -6, 'max': 6, 'palette': palette_diff}, 'Δ VH (dB)', True)

        legend_dict = OrderedDict([
            ('Disminución', '#c0392b'),
            ('Sin cambio', '#fdf6e3'),
            ('Aumento', '#1B7192')
        ])
        Map.add_legend(title='Interpretación de Δ', legend_dict=legend_dict)

        summary_html = f"""
        <div class='success-box'>
          <div class='icon'>📈</div>
          <div class='content'>
            <h3>Visualización generada</h3>
            <p><strong>Período de referencia:</strong> {ref_start_str} → {ref_end_str}</p>
            <p><strong>Período objetivo:</strong> {tar_start_str} → {tar_end_str}</p>
            <p class='next-step'><span>Resumen:</span> Se muestran las diferencias en dB para VV y VH. Valores positivos indican incremento; valores negativos, disminución.</p>
          </div>
        </div>
        """

        display(HTML(summary_html))
        display(Map)
        _status('<strong>Ayuda:</strong> Se recomienda alternar las capas Δ VV y Δ VH para comparar los patrones.', 'info')
run_btn.on_click(run_viz)


## 4️⃣ Detección multitemporal (sequential omnibus)

El módulo multitemporal aplica el test secuencial propuesto por Conradsen et al. (2016) para identificar múltiples cambios dentro de una serie Sentinel-1.

**Cómo proceder**
1. Ejecutar la celda para inicializar el panel.
2. Definir el intervalo completo de estudio (`Inicio` y `Fin`) y el paso temporal (`Intervalo (días)`).
3. Ajustar el nivel de significancia (`α`) según el nivel de confianza deseado (0.01 es una opción conservadora).
4. Decidir si se aplicará el filtro de mediana para suavizar ruido salt-and-pepper.
5. Pulsar `🔬 Ejecutar análisis multitemporal` y esperar a que la barra de progreso llegue al 100 %.

**Mapas producidos**
- **Cambio más reciente:** identifica el momento del último cambio significativo.
- **Primer cambio:** señala el primer evento detectado en la serie.
- **Frecuencia de cambios:** contabiliza cuántos cambios ocurrieron por píxel (útil para ciclos múltiples).
- **Dirección del cambio:** clasifica cada intervalo como aumento, disminución o mixto.

Se recomienda revisar las estadísticas resumidas y, cuando sea posible, contrastar con información de campo o imágenes ópticas complementarias.


In [15]:
#@title <font color=#88B04B>Detección multitemporal (sequential omnibus)</font>

"""Panel interactivo para detección multitemporal (sequential omnibus)."""
import sys, subprocess, traceback as _tb, time
from datetime import date, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import ee
import numpy as np
from collections import OrderedDict

# ================== Funciones auxiliares SAR ==================

def det(im):
    """Calcula el determinante de la matriz de covarianza 2x2 diagonal (VV, VH)."""
    return im.expression('b(0) * b(1)')

def chi2cdf(chi2, df):
    """
    Función de distribución acumulativa Chi-cuadrado para df grados de libertad.
    Usa la función gamma incompleta incorporada en GEE.
    """
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def log_det_sum(im_list, j):
    """Retorna log del determinante de la suma de las primeras j imágenes."""
    im_list = ee.List(im_list)
    sumj = ee.ImageCollection(im_list.slice(0, j)).reduce(ee.Reducer.sum())
    return ee.Image(det(sumj)).log()

def log_det(im_list, j):
    """Retorna log del determinante de la j-ésima imagen."""
    im = ee.Image(ee.List(im_list).get(j.subtract(1)))
    return ee.Image(det(im)).log()

# ================== Test Estadísticos ==================

def pval(im_list, j, m=4.4):
    """
    Calcula -2logRj para una lista de imágenes y retorna el valor p y -2logRj.

    Parámetros:
    - im_list: Lista de imágenes (covariance matrices)
    - j: Índice temporal
    - m: Número equivalente de looks (ENL)

    Retorna: (pv, m2logRj)
    """
    im_list = ee.List(im_list)
    j = ee.Number(j)

    # Ecuación (3.10a) del tutorial Part 3
    m2logRj = (log_det_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(log_det(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(log_det_sum(im_list, j).multiply(j))
               .multiply(-2).multiply(m))

    # valor p usando distribución chi-square con 2 grados de libertad
    pv = ee.Image.constant(1).subtract(chi2cdf(m2logRj, 2))

    return (pv, m2logRj)

def p_values(im_list):
    """
    Pre-calcula el array de valor ps para una lista de imágenes.
    Implementa el algoritmo de la Tabla 3.1 del tutorial Part 3.

    Retorna: Array de valor ps [ell=k...2][j=2...ell]
    """
    im_list = ee.List(im_list)
    k = im_list.length()

    def ells_map(ell):
        """Itera sobre los valores de ell (longitud de la serie)."""
        ell = ee.Number(ell)
        # Cortar la serie desde k-ell+1 hasta k
        im_list_ell = im_list.slice(k.subtract(ell), k)

        def js_map(j):
            """Calcula pval para cada combinación de ell y j."""
            j = ee.Number(j)
            pv1, m2logRj1 = pval(im_list_ell, j)
            return ee.Feature(None, {'pv': pv1, 'm2logRj': m2logRj1})

        # Mapear sobre j=2,3,...,ell
        js = ee.List.sequence(2, ell)
        pv_m2logRj = ee.FeatureCollection(js.map(js_map))

        # Calcular m2logQl = sum(m2logRj) (Ecuación 3.10b)
        m2logQl = ee.ImageCollection(pv_m2logRj.aggregate_array('m2logRj')).sum()
        # valor p para Ql con 2(ell-1) grados de libertad
        pvQl = ee.Image.constant(1).subtract(chi2cdf(m2logQl, ell.subtract(1).multiply(2)))

        # Agregar pvQl al final de la lista de pvs
        pvs = ee.List(pv_m2logRj.aggregate_array('pv')).add(pvQl)
        return pvs

    # Mapear sobre ell = k, k-1, ..., 2
    ells = ee.List.sequence(k, 2, -1)
    pv_arr = ells.map(ells_map)

    return pv_arr

# ================== Filtrado de valor ps para mapas temáticos ==================

def filter_j(current, prev):
    """
    Calcula mapas de cambio; itera sobre índices j del array de p-values.
    Implementa el algoritmo secuencial del tutorial Part 3.
    """
    pv = ee.Image(current)
    prev = ee.Dictionary(prev)
    pvQ = ee.Image(prev.get('pvQ'))
    i = ee.Number(prev.get('i'))
    cmap = ee.Image(prev.get('cmap'))
    smap = ee.Image(prev.get('smap'))
    fmap = ee.Image(prev.get('fmap'))
    bmap = ee.Image(prev.get('bmap'))
    alpha = ee.Image(prev.get('alpha'))
    j = ee.Number(prev.get('j'))

    # Índice del intervalo de cambio
    cmapj = cmap.multiply(0).add(i.add(j).subtract(1))

    # Test: Rj < alpha? AND Ql < alpha? AND en la fila i?
    tst = pv.lt(alpha).And(pvQ.lt(alpha)).And(cmap.eq(i.subtract(1)))

    # Actualizar cmap (intervalo del cambio más reciente)
    cmap = cmap.where(tst, cmapj)

    # Actualizar fmap (frecuencia/número de cambios)
    fmap = fmap.where(tst, fmap.add(1))

    # Actualizar smap solo si estamos en la primera fila
    smap = ee.Algorithms.If(i.eq(1), smap.where(tst, cmapj), smap)

    # Crear banda para bmap y agregarla
    idx = i.add(j).subtract(2)
    tmp = bmap.select(idx)
    bname = bmap.bandNames().get(idx)
    tmp = tmp.where(tst, 1)
    tmp = tmp.rename([bname])
    bmap = bmap.addBands(tmp, [bname], True)

    return ee.Dictionary({
        'i': i, 'j': j.add(1), 'alpha': alpha, 'pvQ': pvQ,
        'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap': bmap
    })

def filter_i(current, prev):
    """
    Organiza el cálculo de mapas de cambio; itera sobre índices de fila del array de p-values.
    """
    current = ee.List(current)
    pvs = current.slice(0, -1)  # Todos excepto el último (que es pvQ)
    pvQ = ee.Image(current.get(-1))
    prev = ee.Dictionary(prev)
    i = ee.Number(prev.get('i'))
    alpha = ee.Image(prev.get('alpha'))
    median = prev.get('median')

    # Aplicar filtro de mediana a pvQ si se solicita (reduce ruido salt-and-pepper)
    pvQ = ee.Algorithms.If(median, pvQ.focalMedian(2.5), pvQ)

    cmap = prev.get('cmap')
    smap = prev.get('smap')
    fmap = prev.get('fmap')
    bmap = prev.get('bmap')

    first = ee.Dictionary({
        'i': i, 'j': 1, 'alpha': alpha, 'pvQ': pvQ,
        'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap': bmap
    })

    result = ee.Dictionary(ee.List(pvs).iterate(filter_j, first))

    return ee.Dictionary({
        'i': i.add(1), 'alpha': alpha, 'median': median,
        'cmap': result.get('cmap'), 'smap': result.get('smap'),
        'fmap': result.get('fmap'), 'bmap': result.get('bmap')
    })

# ================== Loewner Order (Dirección del cambio) ==================

def dmap_iter(current, prev):
    """
    Reclasifica valores en mapas direccionales usando el orden de Loewner.
    Clasifica cambios como:
    - 1: Positivo definido (aumento en ambas polarizaciones)
    - 2: Negativo definido (disminución en ambas polarizaciones)
    - 3: Indefinido (cambio mixto)
    """
    prev = ee.Dictionary(prev)
    j = ee.Number(prev.get('j'))
    image = ee.Image(current)
    avimg = ee.Image(prev.get('avimg'))

    # Diferencia con el promedio provisional
    diff = image.subtract(avimg)

    # Determinar definición positiva/negativa
    posd = ee.Image(diff.select(0).gt(0).And(det(diff).gt(0)))
    negd = ee.Image(diff.select(0).lt(0).And(det(diff).gt(0)))

    bmap = ee.Image(prev.get('bmap'))
    bmapj = bmap.select(j)

    # Crear imagen de clasificación
    dmap = ee.Image.constant(ee.List.sequence(1, 3))
    bmapj = bmapj.where(bmapj, dmap.select(2))  # Por defecto: indefinido
    bmapj = bmapj.where(bmapj.And(posd), dmap.select(0))  # Positivo
    bmapj = bmapj.where(bmapj.And(negd), dmap.select(1))  # Negativo

    bmap = bmap.addBands(bmapj, overwrite=True)

    # Actualizar promedio provisional (algoritmo de medias provisionales)
    i = ee.Image(prev.get('i')).add(1)
    avimg = avimg.add(image.subtract(avimg).divide(i))

    # Reiniciar avimg a imagen actual si hubo cambio
    avimg = avimg.where(bmapj, image)
    i = i.where(bmapj, 1)

    return ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j.add(1), 'i': i})

def change_maps(im_list, median=False, alpha=0.01):
    """
    Calcula mapas temáticos de cambio para una serie temporal de imágenes.

    Parámetros:
    - im_list: Lista de imágenes (ee.List)
    - median: Si True, aplica filtro de mediana a los valor ps de Ql
    - alpha: Nivel de significancia (default: 0.01)

    Retorna:
    ee.Dictionary con:
    - cmap: Intervalo del cambio más reciente
    - smap: Intervalo del primer cambio
    - fmap: Número/frecuencia de cambios
    - bmap: Cambios por intervalo con dirección (Loewner order)
    """
    k = im_list.length()

    # 1. Pre-calcular array de valor ps
    pv_arr = ee.List(p_values(im_list))

    # 2. Filtrar valor ps para crear mapas de cambio
    cmap = ee.Image(im_list.get(0)).select(0).multiply(0)
    bmap = ee.Image.constant(ee.List.repeat(0, k.subtract(1))).add(cmap)
    alpha_img = ee.Image.constant(alpha)

    first = ee.Dictionary({
        'i': 1, 'alpha': alpha_img, 'median': median,
        'cmap': cmap, 'smap': cmap, 'fmap': cmap, 'bmap': bmap
    })

    result = ee.Dictionary(pv_arr.iterate(filter_i, first))

    # 3. Post-procesar bmap para dirección de cambio (Loewner order)
    bmap = ee.Image(result.get('bmap'))
    avimg = ee.Image(im_list.get(0))
    j = ee.Number(0)
    i = ee.Image.constant(1)

    first_dmap = ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j, 'i': i})
    dmap = ee.Dictionary(im_list.slice(1).iterate(dmap_iter, first_dmap)).get('bmap')

    return ee.Dictionary(result.set('bmap', dmap))

# ================== UI para detección multitemporal ==================

# Calcular fechas óptimas basadas en CONFIG si existe
try:
    config_inicio = date.fromisoformat(CONFIG['fecha_inicio'])
    config_fin = date.fromisoformat(CONFIG['fecha_fin'])

    # Parámetros optimizados para arroz: ciclo completo 30 días antes + 150 días después
    period_start_default = config_inicio - timedelta(days=30)
    period_end_default = config_inicio + timedelta(days=150)

    # Ajustar si excede CONFIG fin
    if period_end_default > config_fin:
        period_end_default = config_fin

except (NameError, KeyError):
    # Valores por defecto si no hay CONFIG
    today = date.today()
    period_start_default = date(today.year, 1, 1)
    period_end_default = date(today.year, 5, 31)

# Widgets
period_start = widgets.DatePicker(description='Inicio:', value=period_start_default,
                                  style={'description_width': '100px'})
period_end = widgets.DatePicker(description='Fin:', value=period_end_default,
                                style={'description_width': '100px'})
interval_days = widgets.IntSlider(description='Intervalo (días):', value=12, min=6, max=30, step=6,
                                  style={'description_width': '140px'})
alpha_slider = widgets.FloatSlider(description='Significancia (α):', value=0.01, min=0.001, max=0.1, step=0.001,
                                   style={'description_width': '140px'}, readout_format='.3f')
median_filter = widgets.Checkbox(description='Aplicar filtro de mediana', value=True, indent=False)

run_multitemp_btn = widgets.Button(description='🔬 Ejecutar Análisis Multitemporal',
                                   button_style='info', icon='chart-line',
                                   layout=widgets.Layout(width='auto', margin='10px 0'))

progress_multitemp = widgets.IntProgress(value=0, min=0, max=100, description='0%',
                                        layout=widgets.Layout(width='100%'))
progress_label_multitemp = widgets.HTML("<b>Estado:</b> Listo para iniciar.")
output_multitemp = widgets.Output()

panel_multitemp = widgets.VBox([
    widgets.HTML("<h3 style='color:#1d8043;'>🔬 Detección multitemporal de cambios</h3>"
                "<p>Basado en el test omnibus secuencial de Conradsen et al. (2016)</p>"),
    widgets.HBox([period_start, period_end]),
    interval_days,
    alpha_slider,
    median_filter,
    run_multitemp_btn,
    progress_multitemp,
    progress_label_multitemp,
    output_multitemp
])
panel_multitemp.add_class('multitemp-card')
display(panel_multitemp)

# ================== Ejecutor del análisis multitemporal ==================

def _step_mt(pct, msg, style=''):
    """Helper para actualizar progreso."""
    pct = max(0, min(100, int(pct)))
    progress_multitemp.value = pct
    progress_multitemp.description = f"{pct}%"
    if style in ('success', 'info', 'warning', 'danger'):
        progress_multitemp.bar_style = style
    progress_label_multitemp.value = f"<b>Estado:</b> {msg}"

def run_multitemporal_analysis(_):
    """Ejecuta el análisis multitemporal de cambios."""
    with output_multitemp:
        clear_output(wait=True)

        try:
            # Validar AOI
            _step_mt(5, "Se valida el área de interés...")
            aoi = AOI_FC.geometry()
        except Exception as e:
            display(HTML(f"<div style='color:#c0392b;'><b>Error:</b> No se encontró AOI. "
                        "Ejecuta primero la celda de carga GPKG.</div>"))
            return

        # Validar fechas
        _step_mt(10, "Se validan los parámetros configurados...")
        if period_start.value is None or period_end.value is None:
            display(HTML("<div style='color:#c0392b;'><b>Error:</b> Fechas inválidas.</div>"))
            return

        start_date = period_start.value.strftime('%Y-%m-%d')
        end_date = period_end.value.strftime('%Y-%m-%d')
        interval = interval_days.value
        alpha_val = alpha_slider.value
        use_median = median_filter.value

        # Crear serie temporal
        _step_mt(20, "Se construye la serie temporal...")
        try:
            # Filtrar colección
            s1_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                      .filterBounds(aoi)
                      .filterDate(start_date, end_date)
                      .filter(ee.Filter.eq('instrumentMode', 'IW'))
                      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

            # Preferir ASCENDING
            s1_asc = s1_coll.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
            s1_desc = s1_coll.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
            s1_use = ee.ImageCollection(ee.Algorithms.If(
                s1_asc.size().gt(s1_desc.size()), s1_asc, s1_desc))

            # Seleccionar VV y VH, aplicar clip
            def clip_and_select(img):
                return img.select(['VV', 'VH']).clip(aoi)

            s1_series = s1_use.map(clip_and_select)

            # Convertir a lista
            im_list = s1_series.toList(s1_series.size())
            k = im_list.length().getInfo()

            if k < 3:
                display(HTML(f"<div style='color:#c0392b;'><b>Error:</b> Se necesitan al menos 3 imágenes. "
                            f"Solo se encontraron {k}.</div>"))
                return

            display(HTML(f"<div style='color:#2ca02c;'><b>✓</b> Serie temporal construida: {k} imágenes.</div>"))

        except Exception as e:
            display(HTML(f"<div style='color:#c0392b;'><b>Error</b> construyendo serie: {e}</div>"))
            _tb.print_exc()
            return

        # Ejecutar algoritmo de detección de cambios
        _step_mt(40, "Se ejecuta el test omnibus secuencial...")
        try:
            result = change_maps(im_list, median=use_median, alpha=alpha_val)

            # Extraer mapas
            cmap = ee.Image(result.get('cmap'))  # Cambio más reciente
            smap = ee.Image(result.get('smap'))  # Primer cambio
            fmap = ee.Image(result.get('fmap'))  # Frecuencia de cambios
            bmap = ee.Image(result.get('bmap'))  # Cambios por intervalo con dirección

        except Exception as e:
            display(HTML(f"<div style='color:#c0392b;'><b>Error</b> en análisis: {e}</div>"))
            _tb.print_exc()
            return

        # Crear visualización
        _step_mt(70, "Se generan los mapas de resultados...")
        try:
            # Asegurar que geemap está disponible
            try:
                import geemap
            except:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "geemap>=0.33.2"])
                import geemap

            # Centrar mapa
            center = aoi.centroid(30).coordinates().getInfo()[::-1]
            Map = geemap.Map(center=center, zoom=10, use_google_maps=False)
            Map.add_basemap('SATELLITE')

            # Agregar AOI
            Map.addLayer(aoi, {'color': 'yellow', 'fillColor': '#00000000'}, 'AOI', True, 0.8)

            # Paletas de colores
            palette_temporal = ['black', 'blue', 'cyan', 'yellow', 'orange', 'red']
            palette_loewner = ['#000000', '#3cb371', '#ff4d4d', '#ffa600']

            # Agregar capas
            Map.addLayer(cmap.updateMask(cmap.gt(0)),
                        {'min': 0, 'max': k-1, 'palette': palette_temporal},
                        'Cambio más reciente', False)
            Map.addLayer(smap.updateMask(smap.gt(0)),
                        {'min': 0, 'max': k-1, 'palette': palette_temporal},
                        'Primer cambio', False)
            Map.addLayer(fmap.updateMask(fmap.gt(0)),
                        {'min': 0, 'max': 10, 'palette': palette_temporal},
                        'Frecuencia de cambios', True)

            # Agregar leyenda
            legend_dict = OrderedDict([
                ('Aumento (Pos. Def.)', '#3cb371'),
                ('Disminución (Neg. Def.)', '#ff4d4d'),
                ('Cambio Mixto (Indef.)', '#ffa600')
            ])
            Map.add_legend(title="Dirección del Cambio (Loewner Order)", legend_dict=legend_dict)

            # Calcular estadísticas
            _step_mt(85, "Se calculan las estadísticas complementarias...")
            change_mask = fmap.gt(0)
            area_img = ee.Image.pixelArea().updateMask(change_mask)
            stats = area_img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=aoi,
                scale=30,
                maxPixels=1e11,
                tileScale=8,
                bestEffort=True
            )
            area_cambio_m2 = ee.Number(stats.get('area', 0)).getInfo()
            area_total_m2 = ee.Number(aoi.area()).getInfo()

            area_cambio_ha = area_cambio_m2 / 10000.0
            area_total_ha = area_total_m2 / 10000.0
            pct_cambio = (area_cambio_ha / area_total_ha * 100.0) if area_total_ha > 0 else 0.0

            # Mostrar resultados
            _step_mt(100, "✓ El análisis multitemporal finalizó", style='success')

            summary = f"""
            <div style='background:#30343f; padding:18px; border-radius:10px; border-left:4px solid #9ca3af; color:#f5f6fa;'>
              <h3 style='margin:0 0 12px 0; color:#f5f6fa;'>📊 Resultados del Análisis Multitemporal</h3>
              <table style='width:100%; border-collapse:collapse; color:#f5f6fa;'>
                <tr style='border-bottom:1px solid rgba(255,255,255,0.18);'><td style='padding:6px 0;'><b>Período analizado:</b></td><td style='padding:6px 0;'>{start_date} → {end_date}</td></tr>
                <tr style='border-bottom:1px solid rgba(255,255,255,0.18);'><td style='padding:6px 0;'><b>Número de imágenes:</b></td><td style='padding:6px 0;'>{k}</td></tr>
                <tr style='border-bottom:1px solid rgba(255,255,255,0.18);'><td style='padding:6px 0;'><b>Nivel de significancia:</b></td><td style='padding:6px 0;'>{alpha_val:.3f}</td></tr>
                <tr style='border-bottom:1px solid rgba(255,255,255,0.18);'><td style='padding:6px 0;'><b>Área total:</b></td><td style='padding:6px 0;'>{area_total_ha:,.2f} ha</td></tr>
                <tr style='border-bottom:1px solid rgba(255,255,255,0.18);'><td style='padding:6px 0;'><b>Área con cambios:</b></td><td style='padding:6px 0;'>{area_cambio_ha:,.2f} ha</td></tr>
                <tr><td style='padding:6px 0;'><b>Porcentaje de cambio:</b></td><td style='padding:6px 0;'><b>{pct_cambio:.2f}%</b></td></tr>
              </table>
            </div>
            """
            display(HTML(summary))
            display(Map)

            display(HTML("""
            <div style='margin-top:15px; padding:12px; background:#fff3cd; border-radius:6px;'>
              <b>💡 Interpretación de los mapas:</b>
              <ul style='margin:5px 0;'>
                <li><b>Frecuencia de cambios:</b> Número de cambios detectados en cada píxel (azul=pocos, rojo=muchos)</li>
                <li><b>Primer cambio:</b> Momento del primer cambio detectado (azul=temprano, rojo=tardío)</li>
                <li><b>Cambio más reciente:</b> Momento del último cambio detectado</li>
                <li><b>Dirección:</b> Verde=aumento reflectividad, Rojo=disminución, Naranja=mixto</li>
              </ul>
            </div>
            """))

        except Exception as e:
            display(HTML(f"<div style='color:#c0392b;'><b>Error</b> en visualización: {e}</div>"))
            _tb.print_exc()
            return

# Conectar botón
run_multitemp_btn.on_click(run_multitemporal_analysis)

display(HTML("<hr><p style='text-align:center; color:#666;'>Basado en Conradsen et al. (2016) - "
            "Sequential Omnibus Test for Change Detection in SAR Time Series</p>"))